<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/miniprojet_Jehanne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


28711095 - Jehanne Reboud


# PROJET DM1 : IMDB et YAGO (avec COLAB)


Mini-projet sur la restructuration de données et l'analyse de données.
Voir le moodle pour la date de remise et éventuelles précisions sur le travail à faire.

## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/ens/bdle/DM1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['csvfiles_sample001',
 'csvfiles_sample001.tgz',
 'name.csv.gz',
 'title.csv.gz',
 'yago']

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 41.8 MB/s 


Démarrer la session spark

In [ ]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1634507304790


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = val.split(";")
    for r in tabRequetes:
        if len(r) > 2:
          derniere = spark.sql(r)
    return display(derniere)

## Accès aux données

URL pour accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


#### IMDB
Télécharger les fichiers de IMDB directement sur le cluster databricks, sans passer par votre ordinateur personnel.

##### Sample: échantillon de la base IMDB

In [ ]:
import os
from urllib import request
import tarfile

imdb_dir = drive_dir

url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles_sample001.tgz"
# temp = "/temp"
local_file = imdb_dir + "/csvfiles_sample001.tgz"

os.makedirs(imdb_dir, exist_ok=True)

if(os.path.isfile(local_file)):
  print("file is already downloaded")
else:
  print("downloading from URL: ", url, "save in : ", local_file)
  request.urlretrieve(url, local_file)
  os.chdir(imdb_dir)
  tar = tarfile.open(local_file)
  tar.extractall()
  tar.close()

# Liste des fichiers de IMDB
dir = imdb_dir + "/csvfiles_sample001/"
os.listdir(dir)

file is already downloaded


['title.csv',
 'cast_info.csv',
 'movie_info.csv',
 'person_info.csv',
 'name.csv',
 'movie_companies.csv',
 'company_name.csv',
 'role_type.csv',
 'kind_type.csv',
 'company_type.csv',
 'info_type.csv']

##### Données complètes: name_all et title_all
Télécharger le fichier complet des noms de personnes et les titres de films

In [ ]:
import os
def load_IMDB_file(file):
  if(os.path.isfile( imdb_dir + "/" + file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles/" + file
    print("downloading from URL: ", url, "save in : " + imdb_dir + "/" + file)
    request.urlretrieve(url , imdb_dir + "/"  + file)

load_IMDB_file("name.csv.gz")
load_IMDB_file("title.csv.gz")

# Liste des fichiers de IMDB
os.listdir(imdb_dir)

name.csv.gz is already stored
title.csv.gz is already stored


['csvfiles_sample001',
 'csvfiles_sample001.tgz',
 'name.csv.gz',
 'title.csv.gz',
 'yago']

In [ ]:
# name_all

schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name_all = spark.read.csv(path = imdb_dir + "/name.csv.gz", schema = schema_name)
name_all.createOrReplaceTempView("NameAll")
display(name_all)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,3343,"Abela, Mike",None,NaN,m,A1452,M214,A14,61f733c0298a7cb5a461fd787a655d70
1,446,"A., David",None,NaN,m,A313,D13,A,cf45e7b42fbc800c61462988ad1156d2
2,126,"-Alverio, Esteban Rodriguez",None,NaN,m,A4162,E2315,A416,f5c410bff6839b545d04c531f776e8f2
3,1678,"Abbas, Athar",None,NaN,m,A1236,A3612,A12,cf230f6ed718a330dc688c20b8d741d3
4,3610,"Aberer, Leo",None,NaN,m,A164,L16,A16,05684f16b84db387e613d3de57eca5b7
...,...,...,...,...,...,...,...,...,...
95,3211,"Abeghraz, Rachid",None,NaN,m,A1262,R2312,None,2b054445d528ca9f6994adedd9db9722
96,2129,"Abbott, Percy",None,NaN,m,A1316,P6213,A13,7992faa0fc9acd53bebbb68549f9360a
97,1167,"Abacan, Jose Mari",None,NaN,m,A1252,J2561,A125,06f8d8a8e7df8e7887ea7e7c7a671b8c
98,331,"5, Channel",None,NaN,m,C54,None,None,4db55ab39bb71cbe53eda4eacb768705


In [ ]:
# title_all

schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """
title_all = spark.read.csv(path = imdb_dir + "/title.csv.gz", schema = schema_title)
title_all.createOrReplaceTempView("TitleAll")
display(title_all)

,id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_id,season_nr,episode_nr,series_years,md5sum
0,80889,(#1.66),None,7,1980.0,NaN,None,80815,1.0,66.0,None,4de45f35edf0b753c54ab72dcbe68bb5
1,5156,Josie Duggar's 1st Shoes,None,7,2010.0,NaN,J2326,5022,4.0,13.0,None,8d492850166137b550ceb0e9a5b46086
2,197772,(#2.8),None,7,1962.0,NaN,None,197751,2.0,8.0,None,152b0f7213dd467760823b0d6387a6ec
3,111913,(2012-09-13),None,7,2012.0,NaN,None,111095,NaN,NaN,None,daf144630ef1ab1cd5d1d028b406cdcf
4,117556,(#1.1042),None,7,NaN,NaN,None,117506,1.0,1042.0,None,be62adfea75e456d47ddab5c68ac96b0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,74371,American Justice,None,2,2004.0,NaN,A5625,None,NaN,NaN,2004-????,d0d9d994eb34142614894127dc98f750
96,5392,(#1.8),None,7,2011.0,NaN,None,5383,1.0,8.0,None,66aa6123117ad570cff22a0961cde934
97,115268,The Boy Next Door,None,7,2008.0,NaN,B5236,114979,13.0,18.0,None,2da5276828def251a95a367c5a74359a
98,120056,(#2.9),None,7,2009.0,NaN,None,120036,2.0,9.0,None,a44636910a50bf5613640b4c80ff3e37


#### YAGO
Télécharger les fichiers de YAGO

In [ ]:
YAGO_dir = drive_dir + "/yago"

os.makedirs(YAGO_dir, exist_ok=True)

def yagoLoad(file):
  url = PUBLIC_DATASET + "/yago/yago4_2020/" + file
  local_file = YAGO_dir + "/" + file

  if(os.path.isfile( YAGO_dir + "/" + file)):
    print(file, "is already stored ")
  else:
    print("downloading from URL: ", url, "save in : ", local_file)
    request.urlretrieve(url, local_file)
    
yagoLoad("yago-actor-facts.snappy.orc")
yagoLoad("yago-person.snappy.orc")

# Liste des fichiers de YAGO
os.listdir(YAGO_dir)

yago-actor-facts.snappy.orc is already stored 
yago-person.snappy.orc is already stored 


['yago-actor-facts.snappy.orc', 'yago-person.snappy.orc']

##### Les personnes dans Yago

In [ ]:
yagoPerson = spark.read.orc(path = YAGO_dir + "/yago-person.snappy.orc")
display(yagoPerson)

,subject
0,<http://yago-knowledge.org/resource/Heywood_L....
1,<http://yago-knowledge.org/resource/Ramanichan...
2,<http://yago-knowledge.org/resource/Yegor_Kiry...
3,<http://yago-knowledge.org/resource/Patrick_J....
4,<http://yago-knowledge.org/resource/Basilio_J....
...,...
95,<http://yago-knowledge.org/resource/Billy_Gowers>
96,<http://yago-knowledge.org/resource/Simon_Kain...
97,<http://yago-knowledge.org/resource/K._Sukumaran>
98,<http://yago-knowledge.org/resource/Harry_Melr...


##### Les faits des acteurs dans Yago

In [ ]:
actorFacts = spark.read.orc(path = YAGO_dir + "/yago-actor-facts.snappy.orc")
display(actorFacts)

,subject,property,object
0,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/alumniOf>,<http://yago-knowledge.org/resource/Academy_of...
1,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/nationality>,<http://yago-knowledge.org/resource/Canada>
2,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/birthPlace>,<http://yago-knowledge.org/resource/Toronto>
3,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Actor>
4,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Songwriter>
...,...,...,...
95,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Opera_singer>
96,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Actor>
97,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/birthDate>,"""1887-12-12""^^<http://www.w3.org/2001/XMLSchem..."
98,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/deathPlace>,<http://yago-knowledge.org/resource/Copenhagen>


## Définir les tables IMDB

Le dossier contenant les fichiers csv de IMDB

In [ ]:
dir = drive_dir + "/csvfiles_sample001/"

### Les tables initiales

#### Title
La table Title affecte un identifiant id à un film

In [ ]:
schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """

title = spark.read.csv(path = dir + "title.csv", schema = schema_title).persist()
title.createOrReplaceTempView("Title")
display(title)


,id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_id,season_nr,episode_nr,series_years,md5sum
0,7532,"Beautiful, Married & Missing",None,7,2009.0,NaN,B3145,7383,NaN,NaN,None,890adf0e8ff5ba69b8f6dc7b0dce84e7
1,160794,Self-Awareness as a Soldier! Strength Lies in ...,None,7,1995.0,NaN,S4165,160772,1.0,38.0,None,eb32612a744424c3b79a8959c48aa89b
2,20232,Baby Manning,None,7,1998.0,NaN,B152,20037,NaN,NaN,None,825389d4e35f378f950238020942782e
3,16604,The Next First Lady?/Here Comes the Girls/Toni...,None,7,1999.0,NaN,N2316,15711,31.0,20.0,None,6385686401009aa7277c38cea7db489c
4,189974,(#1.6),None,7,2000.0,NaN,None,189966,1.0,6.0,None,2b97f31e49e4cea3da14e1d1292915d2
...,...,...,...,...,...,...,...,...,...,...,...,...
95,162317,Candi Hearts,None,7,2005.0,NaN,C5363,162314,2.0,13.0,None,8e3f9ff72d9bbab40d0720c0ad90af57
96,152926,2003: Uplate 2,None,7,2003.0,NaN,U143,152916,3.0,5.0,None,56f310ed6b6996c957bbd2f3ce133e36
97,134904,(2010-04-13),None,7,2010.0,NaN,None,134557,NaN,NaN,None,4ff4c52967760bcb636f8c7ba7af253b
98,60070,Amandus und der Clown,None,7,1970.0,NaN,A5325,60069,2.0,2.0,None,04b4356dea3ca5041986eadfd366a139


#### Kind_Type et Info_Type
* La table Kind_type
* La table Info_type indique la nature d'un attribut d'information. 

Sert dans Movie_Info, Person_Info

In [ ]:
#============
# Kind_Type
#============
schema_kind_type = "id INT, kind STRING"

kind_type = spark.read.csv(path = dir + "kind_type.csv", schema = schema_kind_type).persist()

kind_type.createOrReplaceTempView("Kind_Type")
# kind_type.printSchema()
# display(kind_type)



#==============
# Info_Type
#==============
schema_info_type = "id INT, info STRING"

info_type = spark.read.csv(path = dir + "info_type.csv", schema = schema_info_type).persist()

info_type.createOrReplaceTempView("Info_Type")
#info_type.printSchema()

display(info_type)

,id,info
0,1,runtimes
1,2,color info
2,3,genres
3,4,languages
4,5,certificates
...,...,...
95,96,other literature
96,97,mpaa
97,98,plot
98,99,votes distribution


#### Movie_info

In [ ]:
#==============
# Movie_Info
#==============
schema_movie_info = """
    id int,
    movie_id int,
    info_type_id int,
    info string,
    note string
"""

movie_info = spark.read.csv(path = dir + "movie_info.csv", schema = schema_movie_info).persist()

movie_info.createOrReplaceTempView("Movie_Info")
#movie_info.printSchema()

display(movie_info)

,id,movie_id,info_type_id,info,note
0,11658949,13,16,Italy:29 March 2012,None
1,11659887,434,16,Netherlands:7 January 2013,None
2,11661227,1680,16,USA:11 May 2011,None
3,11661228,1680,16,USA:11 May 2011,None
4,11661771,1841,16,Spain:20 March 2005,None
...,...,...,...,...,...
95,11671027,10280,16,UK:25 February 1984,None
96,11671080,10331,16,UK:11 February 1984,None
97,11671391,10750,16,USA:29 August 2009,None
98,11672626,11564,16,Australia:14 July 2012,None


In [ ]:
%%sql
select * from Movie_Info where movie_id = 3981	

,id,movie_id,info_type_id,info,note
0,2873429,3981,2,Color,None
1,8320156,3981,18,"Action Concept Studios, Hürth-Kalscheuren, Nor...",(studio)
2,4162328,3981,8,Germany,None
3,8320157,3981,18,"Hürth, North Rhine - Westphalia, Germany",(studio)
4,7021739,3981,4,German,None
5,8320158,3981,18,"North Rhine - Westphalia, Germany",None
6,11663905,3981,16,Germany:22 October 2008,None
7,11663906,3981,16,Hungary:30 June 2009,None


In [ ]:
%%sql
select * from Movie_Info where note is not null

,id,movie_id,info_type_id,info,note
0,8320156,3981,18,"Action Concept Studios, Hürth-Kalscheuren, Nor...",(studio)
1,8320157,3981,18,"Hürth, North Rhine - Westphalia, Germany",(studio)
2,8981275,8413,1,USA:60,(with commercials)
3,10257857,8413,7,CAM:Panaflex Camera and Lenses by Panavision,(as Panavision®)
4,10257858,8413,7,"LAB:Alpha Cine Service, Vancouver, Canada",(film lab) (as Alpha Cine)
...,...,...,...,...,...
95,2074120,233641,98,Cole Barker's abduction by kidnapping causes t...,KGF Vissers
96,2074299,233923,98,Carrington-Smythe sends Barry and Paul to a sp...,don @ minifie-1
97,2074548,238603,98,This episode features goofy foreign commercial...,Anonymous
98,2074922,251573,98,"In ""Operation: A.F.L.O.A.T. (Agents Feared Los...",Anonymous


#### Cast_Info

In [ ]:
#==============
# Cast_Info
#==============
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

display(cast_info)

,id,person_id,movie_id,person_role_id,note,nr_order,role_id
0,16255404,2202581,434,2671901.0,None,9.0,2
1,31519800,3558778,434,NaN,None,NaN,9
2,16668206,2256708,434,2264435.0,None,2.0,2
3,1148671,159670,434,271127.0,None,1.0,1
4,18308464,2464312,434,2922782.0,None,NaN,2
...,...,...,...,...,...,...,...
95,32328582,3641142,3342,NaN,(audience coordinator),NaN,10
96,22749468,2939379,3342,NaN,(associate producer),NaN,3
97,32525393,3660322,3342,NaN,(fashion stylist),NaN,10
98,23245345,2985931,3342,NaN,(producer),NaN,3


#### Role_type

In [ ]:
#=============
# Role_Type
#=============
schema_role_type = "id INT, role STRING"

role_type = spark.read.csv(path = dir + "role_type.csv", schema = schema_role_type).persist()

role_type.createOrReplaceTempView("Role_Type")
#role_type.printSchema()

display(role_type)

,id,role
0,1,actor
1,2,actress
2,3,producer
3,4,writer
4,5,cinematographer
5,6,composer
6,7,costume designer
7,8,director
8,9,editor
9,10,miscellaneous crew


#### Name
La table Name affecte un identifiant id à une personne

In [ ]:
#==============
# Name 
#==============
schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name = spark.read.csv(path = dir + "name.csv", schema = schema_name).persist()

name.createOrReplaceTempView("Name")
#name.printSchema()

display(name)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,333,"5, Johnny",None,NaN,m,J5,None,None,c845df6a62d46f897c7620f00f1e97a5
1,339,50 Cent,None,NaN,m,C53,None,None,faba8d4ac1b5880ef64a5de3ea7d1fa9
2,400,"A El Rahim, Ahmed",None,NaN,m,A4653,A5346,A465,9698c31ec0eb8b38172ae89e8c67b028
3,457,"A., Kodanda Rami Reddy",None,NaN,m,A2353,K3536,A,64223f8728574d3dce39126c365b5dc9
4,471,"A., Randy",None,NaN,m,A653,R53,A,b5301fd59f93e50ab8893e9b80e3a35a
...,...,...,...,...,...,...,...,...,...
95,7766,"Adamczak, Alfred",None,NaN,m,A3524,A4163,A352,9f810e511b9abfccaae85efedec24dec
96,7822,"Adamec, Jirí",None,NaN,m,A3526,J6352,A352,98cb8be3aa5048fab78c855c6a16abb8
97,7825,"Adamec, Lukás",None,NaN,m,A3524,L2352,A352,5095421885465f116cf77777b010ebcc
98,7915,"Adamo, Antonio",None,NaN,m,A3535,A5353,A35,703b6dd48dad5980b9d12e6b79bbb989


#### Person_Info

In [ ]:
#==============
# Person_Info
#==============
schema_person_info = """
  id int,
  person_id int,
  info_type_id int,
  info string,
  note string
"""

person_info = spark.read.csv(path = dir + "person_info.csv", schema = schema_person_info).persist()
person_info.createOrReplaceTempView("Person_Info")

display(person_info)

,id,person_id,info_type_id,info,note
0,302,333,33,(2012) Portlandia,None
1,303,333,26,"Martini-Connally, John",None
2,304,333,20,"Los Angeles, California, USA",None
3,305,333,17,Has worked & studied alongside studio engineer...,None
4,306,333,17,Bachelors in Digital Media Production from the...,None
...,...,...,...,...,...
95,397,339,36,"""Entertainment Weekly"" (USA), 23 February 2003...",None
96,398,339,26,"III, Curtis James Jackson",None
97,399,339,17,Is involved in a feud with fellow rapper 'Ja R...,None
98,400,339,17,Is signed to 'Eminem' (qv)'s Shady Records imp...,None


#### Movie_Companies

In [ ]:
#==============
# Movie_Companies
#==============
schema_movie_companies = """
    id int,
    movie_id int,
    company_id int,
    company_type_id int,
    note string
"""

movie_companies = spark.read.csv(path = dir + "movie_companies.csv", schema = schema_movie_companies).persist()

movie_companies.createOrReplaceTempView("Movie_Companies")
#movie_companies.printSchema()

display(movie_companies)

,id,movie_id,company_id,company_type_id,note
0,1318,3342,216,1,(2000) (USA) (TV)
1,1319,3342,217,1,(200?) (Japan) (TV)
2,1454,3981,46,1,(2009) (Hungary) (TV)
3,2889,8413,423,1,(2007) (Japan) (DVD)
4,2890,8413,49,1,(1987-1990) (USA) (TV) (original airing)
...,...,...,...,...,...
95,1297300,85610,789,2,None
96,1297301,85610,72715,2,None
97,1297403,85745,21107,2,None
98,1297588,86268,72735,2,None


#### Company_Name

In [ ]:
schema_company_name = """
    id int,
    name string,
    country_code string,
    imdb_id int,
    name_pcode_nf string,
    name_pcode_sf string,
    md5sum string
"""

company_name = spark.read.csv(path = dir + "company_name.csv", schema = schema_company_name)
company_name.createOrReplaceTempView("Company_Name")
#company_name.printSchema()

display(company_name)

,id,name,country_code,imdb_id,name_pcode_nf,name_pcode_sf,md5sum
0,1,E! Entertainment Television,[us],NaN,E5363,E5363,ddc1ca3453a33526ad931106cb60d004
1,2,YouTube,[us],NaN,Y31,Y312,3bd77ab33669d8883d345ee754928caf
2,4,5*,[gb],NaN,None,G1,2c0b57f4db6ba181ca8e7414284e9d35
3,5,Canadian Television (CTV),[ca],NaN,C5353,C5353,7bde1000f06906373c3ae6a48d981511
4,6,Columbia Broadcasting System (CBS),[us],NaN,C4516,C4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
...,...,...,...,...,...,...,...
95,129,KIKA,[de],NaN,K2,K23,561629423916c2a71f4aa5863f9388ec
96,132,Comcast,[us],NaN,C523,C5232,65403fe5213a23ff74526488eee7b1d1
97,133,NHK,[jp],NaN,N2,N21,b43377605ac37f1e62dd047e04f1c135
98,136,Turner Broadcasting System (TBS),[us],NaN,T6561,T6561,c6bcb939ad73a310ad2c3e529938eb15


#### Company_type

In [ ]:
#==============
# Company_Type
#==============
schema_company_type = "id INT, kind STRING"

company_type = spark.read.csv(path = dir + "company_type.csv", schema = schema_company_type).persist()
company_type.createOrReplaceTempView("Company_Type")
#company_type.printSchema()

display(company_type)

,id,kind
0,1,distributors
1,2,production companies
2,3,special effects companies
3,4,miscellaneous companies


### Detail sur les personnes et les oeuvres

#### TitleDetail

In [ ]:
%%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
;

select * 
from TitleDetail t
order by  t.production_year desc, t.title
limit 100;

,id,title,production_year,kind,property,value
0,2310404,Star Trek 3,2016,movie,countries,USA
1,2310404,Star Trek 3,2016,movie,languages,English
2,2310404,Star Trek 3,2016,movie,genres,Action
3,2310404,Star Trek 3,2016,movie,genres,Sci-Fi
4,2310404,Star Trek 3,2016,movie,genres,Adventure
...,...,...,...,...,...,...
95,1901169,Frank,2014,movie,budget,"$18,000,000"
96,1901169,Frank,2014,movie,genres,Drama
97,1901169,Frank,2014,movie,locations,"New York City, New York, USA"
98,1901169,Frank,2014,movie,locations,"Marbella, Málaga, Andalucía, Spain"


Mémoriser le résultat de TitleDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table TitleDetail

""


#### NameDetail

In [ ]:
%%sql
create or replace temp view NameDetail as
select n.id, n.name, n.gender, i.info as property, p.info as value
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
;
 
select * 
from NameDetail n
where n.name like 'Smi%'
order by n.name, n.property
limit 100;

,id,name,gender,property,value
0,2544489,"Smid, Coosje",f,trivia,Daughter of 'Ernst Daniël Smid' (qv) and Rosem...
1,1454381,"Smid, Ernst Daniël",m,birth date,6 May 1953
2,1454381,"Smid, Ernst Daniël",m,birth notes,"Enschede, Overijssel, Netherlands"
3,1454381,"Smid, Ernst Daniël",m,magazine cover photo,"""Ncrv Gids"" (Netherlands), 17 April 1999, Iss. 16"
4,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Clayton in the Dutch ver...
...,...,...,...,...,...
95,1454554,"Smiley, Tavis",m,magazine cover photo,"""Black Enterprise"" (USA), March 2002, Vol. 32,..."
96,1454554,"Smiley, Tavis",m,trivia,"Is a member of Kappa Alpha Psi Fraternity, Inc..."
97,1454554,"Smiley, Tavis",m,trivia,Distant cousin of 'T.D. Jakes' (qv) and 'James...
98,1454554,"Smiley, Tavis",m,where now,"(2006) Release of his book, ""What I Know for S..."


Mémoriser le résultat de NameDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table NameDetail;

""


## QUESTIONS

### Exercice 1: Extraction d'information

Définir ces relations apportant des informations structurées sur les personnes. Le schéma de chaque table est donné à titre indicatif et vous pouvez l'adapter et le compléter si nécessaire.

#### Question 1 : Salary
 **Salary** (id, name, titleId, titleName, salary, extra, currency) avec
  - *salary* est le salaire perçu par une personne pour un film
  - *frequency* indique si le salaire est pour le film, un épisode de la série, une semaine de tournage, etc...
  - *extra* est un complément de salaire (exple pourcentage sur le revenu du film)
  - *currency* est la devise du salaire
 

Voir la propriété *salary history*. Le champ *titleId* peut être nul si aucune correspondance n'est trouvée dans la table TitleDetail. Le cas échéant, essayer de trouver une correspondance avec le fichier *title_all* contenant tous les films.

In [ ]:
%%sql

select * from NameDetail
where property = 'salary history';

,id,name,gender,property,value
0,1989,"Abbott, Bud",m,salary history,"_Rio Rita (1942)_ (qv)::$75,000"
1,1989,"Abbott, Bud",m,salary history,"_One Night in the Tropics (1940)_ (qv)::$17,500"
2,1989,"Abbott, Bud",m,salary history,"_""The Abbott and Costello Show"" (1952)_ (qv)::..."
3,1989,"Abbott, Bud",m,salary history,"_In the Navy (1941)_ (qv)::$25,000 + 5% of pro..."
4,1989,"Abbott, Bud",m,salary history,"_Buck Privates (1941)_ (qv)::$25,000 + 5% of p..."
...,...,...,...,...,...
95,311734,"Cooper, Gary",m,salary history,_Beau Sabreur (1928)_ (qv)::$150/week
96,311734,"Cooper, Gary",m,salary history,_A Farewell to Arms (1932)_ (qv)::$2500/week
97,311734,"Cooper, Gary",m,salary history,_Nevada (1927)_ (qv)::$150/week
98,311734,"Cooper, Gary",m,salary history,_Arizona Bound (1927)_ (qv)::$150/week


# Q1: Salary 

Pour construire la Table Salary:

-
on prend d'abord les valeurs des id et noms (name) des Personnes de la table Namedetail ayant la propriété 'salary history' que l'on appelle nameId et name.

-
Le titleName, titleYear, currency ,salary, frequency et extra sont contenus dans value de Namedetail avec la propriété 'salary history' tous exprimés dans une chaine de caractères de type suivante :	_"Lois & Clark: The New Adventures of Superman" (1993)_ (qv)::$60,000/episode.
Pour extraire ces morceaux de value on utilise les fonctions regexp_extract (et trim pour enlever les espaces en aval et en amont de trop.)

-
Les Id des titres correspondants aux titleName sont en principe  contenus dans TitleDetail. Pour les retrouver,on fait une jointure entre Titledetail et Namedetail sur les  noms de titres (titleName) extraits de value de Namedetail.
Les id correspondant au noms titleName ne sont pas tous présents dans la table Titledetail. Or Titledetail est un sous échantillon de TitleAll.
Donc si les id sont présents (not null) alors on les prend sinon (si il n'y a pas de correspondance) on selectionne dans la table TitleAll les id manquant qui correspondent aux titleName. 




In [ ]:
%%sql

create or replace temp view Salary as
select n.id as nameId, n.name, 
trim(regexp_extract(n.value, '_"?([^\\(\\"]+)"? \\(', 1)) as titleName,
regexp_extract(n.value, '_([^\\(]+) \\(([0-9]*).*\\)_', 2) as titleYear, 

if (n.id is not null,n.id,
  (select first(ta.id) from TitleAll  ta where trim(regexp_extract(n.value, '_"?([^\\(\\"]+)"? \\(', 1)) = ta.title) 
) as titleId,

regexp_extract(n.value, '[^:]+::([^0-9]*)([0-9,m]+)', 2) as salary,
regexp_extract(n.value, '[^:]+::([^0-9]*)([0-9,m]+).*(/|per )([^ +]+)*', 4) as frequency,

if(regexp_extract(n.value, '[^:]+::([^0-9]*)([0-9,m]+)', 1)='',
    regexp_extract(n.value, '[^:]+::([^0-9]*)([0-9,m]+) ([^ +]+)', 3), 
    regexp_extract(n.value, '[^:]+::([^0-9]*)(([0-9,]+|(m ))*)', 1)) as currency,
  
regexp_extract(n.value, '[^:]+::([^0-9]*)([0-9,m]+)(([^+]*\\+)|(.*plus))([^\\)]*)', 6) as extra
from NameDetail n

left join TitleDetail on trim(regexp_extract(n.value, '_"?([^\\(\\"]+)"? \\(', 1)) = TitleDetail.title 
where n.property='salary history';

select * from Salary;

,nameId,name,titleName,titleYear,titleId,salary,frequency,currency,extra
0,2037895,"Garner, Jennifer",13 Going on 30,2004,2037895,"3,000,000",,$,
1,1314543,"Robbins, Peter",A Charlie Brown Christmas,1965,1314543,125,,$,
2,1573563,"Travolta, John",A Civil Action,1998,1573563,"20,000,000",,$,
3,112323,"Beaudine, William",A High Diver's Last Kiss,1918,112323,100,week,$,
4,557318,"Gibson, Hoot",A Man's Land,1932,557318,"16,000",,$,
...,...,...,...,...,...,...,...,...,...
95,1618393,"Verhoeven, Paul",Basic Instinct,1992,1618393,"5,000,000",,$,
96,1618393,"Verhoeven, Paul",Basic Instinct,1992,1618393,"5,000,000",,$,
97,1618393,"Verhoeven, Paul",Basic Instinct,1992,1618393,"5,000,000",,$,
98,1618393,"Verhoeven, Paul",Basic Instinct,1992,1618393,"5,000,000",,$,


#### Question 2 : Height
 **Height**(id, name, heightInCm) qui recense la taille des personnes de manière exploitable, en les reconvertissant en tailles en cm, alors que les entrées de la base sont  en plusieurs unités.

In [ ]:
%%sql

select * from NameDetail
where property = 'height';

,id,name,gender,property,value
0,1521,"Abatantuono, Diego",m,height,183 cm
1,1989,"Abbott, Bud",m,height,"5' 8"""
2,3482,"Abello, Jorge Enrique",m,height,182 cm
3,3892,"Abineri, Daniel",m,height,"6' 0"""
4,4318,"Abraham, Arthur",m,height,178 cm
...,...,...,...,...,...
95,74380,"Azar, Pablo",m,height,179 cm
96,77842,"Bacon, Alexandre",m,height,176 cm
97,77908,"Bacon, Lloyd",m,height,"5' 10"""
98,79357,"Bagby, Larry",m,height,6'


#Q2: Height

Pour construire la Table Height:


On prend d'abord les colonnes des id et noms (name) des Personnes de la table Namedetail ayant la propriété 'height'.

Comme visualisé dans la table précédente, la hauteur contenue dans value de Namedetail avec la propriété 'height',  peut être exprimée sous les 4 formes suivantes. On cherche à toutes les convertir en centimètre et à les afficher dans heightInCm.


-cm : si on localise 'cm' on extrait la valeur avant et on la laisse intacte
-

-feet' : si il n'y a pas de inch (pas de ") on prend la valeur avant les ' que l'on converti en nombre entier (cast()) que l'on multiplie par 30.48 pour convertir en cm
-

-feet' inch 1/2" :si on localise 1/2 le calcul est : ( valeur avant '+ (valeur avant"/12)+ 1/2) *30.48 = valeur en cm

-
 feet'inch" :sinon ( feet '+ (inch"/12)) *30.48 = valeur en cm

In [ ]:
%%sql

create or replace temp view Height as
select n.id, n.name, 
-- cm
if (locate('cm', n.value)>0, regexp_extract(n.value, "(.+) cm", 1),
-- feets' 
if (locate('"', n.value)=0 , cast(regexp_extract(n.value, "(.+)'", 1) as int)*30.48,
--feet' inch 1/2"
if (locate('1/2', n.value)>0, (cast(regexp_extract(n.value, "(.+)' ([0-9]+)", 1) as int)+ (cast(regexp_extract(n.value, "(.+)' ([0-9]+)", 2) as int)+0.5)/12)*30.48,
-- feet'inch"
(cast(regexp_extract(n.value, "(.+)' ?([0-9]+)", 1) as int)+ cast(regexp_extract(n.value, "(.+)' ?([0-9]+)", 2) as int)/12)*30.48))) as height_in_cm
from NameDetail n
where property = 'height';


select * from Height;




,id,name,height_in_cm
0,1521,"Abatantuono, Diego",183
1,1989,"Abbott, Bud",172.72
2,3482,"Abello, Jorge Enrique",182
3,3892,"Abineri, Daniel",182.88
4,4318,"Abraham, Arthur",178
...,...,...,...
95,74380,"Azar, Pablo",179
96,77842,"Bacon, Alexandre",176
97,77908,"Bacon, Lloyd",177.79999999999998
98,79357,"Bagby, Larry",182.88


#### Question 3 : Magazine

 **MagazineCover**(id, name, magazine, country, day, month, year) donnant le nom du magazine dans lequel la personne est sur la couverture, avec le pays du magazine et la date de parution. Voir la propriété *magazine cover photo*.

In [ ]:
%%sql

select * from NameDetail
where property = 'magazine cover photo';

,id,name,gender,property,value
0,4662,"Abramoff, Jack",m,magazine cover photo,"""Time"" (USA), 16 January 2006"
1,21326,"Albers, Hans",m,magazine cover photo,"""Der Spiegel"" (Germany), 14 February 2000"
2,21326,"Albers, Hans",m,magazine cover photo,"""Grosse Österreich Illustrierte"" (Austria), 25..."
3,21326,"Albers, Hans",m,magazine cover photo,"""Das Ufer"" (West Germany), April 1951"
4,21326,"Albers, Hans",m,magazine cover photo,"""Quick"" (West Germany), 25 February 1951, Iss. 8"
...,...,...,...,...,...
95,169633,"Botham, Ian",m,magazine cover photo,"""Private Eye"" (UK), 22 August 1986, Iss. 644"
96,169633,"Botham, Ian",m,magazine cover photo,"""Wisden Cricket Monthly"" (UK), May 1986"
97,169633,"Botham, Ian",m,magazine cover photo,"""Wisden Cricket Monthly"" (UK), August 1982"
98,169633,"Botham, Ian",m,magazine cover photo,"""Wisden Cricket Monthly"" (UK), October 1981"


# Q3: Magazine
Pour construire la Table Magazine:


On prend d'abord les colonnes des id et noms (name) des Personnes de la table Namedetail ayant la propriété 'magazine cover photo'.

Comme visualisé dans la table précédente, le nom du magazine (magazine), le pays (country),  et la date (day, month, year) sont contenu dans des chaines de caractères de value.


-magazine: on extrait le nom entre " " (1ère cellule)
-

-country : on extrait entre parenthèses (3 ème cellule)
-

date: ils n'apparaissent pas dans chaque chaine de value,

jours : extraction 1 ou 2 chiffres

mois : lettres après jour

année: 4 chiffres



In [ ]:
%%sql

create or replace temp view MagazineCover as
select n.id, n.name,

-- magazine
regexp_extract(n.value, '\\"(.+)\\"( \\(([^\\)]*)\\))*', 1) as magazine,
-- country 
regexp_extract(n.value, '\\"(.+)\\"( \\(([^\\)]*)\\))*', 3) as country,
-- date : jour,mois,année
regexp_extract(n.value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?([0-9]{1,2}) ', 6) as day,
regexp_extract(n.value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]*([a-zA-Z\\-/]*)( [0-9]|$|,)', 6) as month,
regexp_extract(n.value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]{0,2}[a-zA-Z \\-/]*[^a-zA-Z]{0,2}(, )?([0-9]{4})', 7) as year

from NameDetail n
where property = 'magazine cover photo';

select * from MagazineCover;

,id,name,magazine,country,day,month,year
0,4662,"Abramoff, Jack",Time,USA,16,January,2006
1,21326,"Albers, Hans",Der Spiegel,Germany,14,February,2000
2,21326,"Albers, Hans",Grosse Österreich Illustrierte,Austria,25,July,1953
3,21326,"Albers, Hans",Das Ufer,West Germany,,April,1951
4,21326,"Albers, Hans",Quick,West Germany,25,February,1951
...,...,...,...,...,...,...,...
95,169633,"Botham, Ian",Private Eye,UK,22,August,1986
96,169633,"Botham, Ian",Wisden Cricket Monthly,UK,,May,1986
97,169633,"Botham, Ian",Wisden Cricket Monthly,UK,,August,1982
98,169633,"Botham, Ian",Wisden Cricket Monthly,UK,,October,1981


#### Question 4 : Trivia

Deux tables détaillant les anecdotes connues sur des personnes, voir les valeurs de la propriété *trivia* d'une personne : 
- **TitleTrivia** (fromPersonId, fromPersonName, toTitleId, toTitleName, triviaText)
- **NameTrivia** (fromPersonId,  fromPersonName, toPersonId, toPersonName, triviaText)
    
Indications :  

Pour le champ toTitleId dans TitleTrivia : si le film n'est pas référencé dans la table TitleDetail, essayer de trouver une correspondance dans le fichier contenant tous les films (title_full).

Pour le champ toPersonId dans NameTrivia : si la personne n'est pas référencée dans la table NameDetail, essayer de trouver une correspondance dans le fichier contenant toutes les personnes (name_full).

# Q4: NameTrivia

Pour construire la Table NameTrivia:

-
On prend d'abord les valeurs des id (fromPersonId) et noms (fromPersonName) des Personnes de la table Namedetail ayant la propriété 'trivia' et qui contienne ' (qv)' dans leur colonne value. En effet, dans la colonne value, si une personne est citée (toPersonName) alors le nom est suivi de ' (qv)'.On prend aussi la colonne de value que l'on renomme triviaText.

-
ToPersonName est contenu dans value entre quote et suivi de' (qv)'. ex: 'Youssef Qissi' (qv).
Pour extraire ces morceaux de value entre ' ' et avant (qv) on utilise les fonctions regexp_extract (et trim pour enlever les espaces en aval et en amont de trop.)


-
Les ToPersonId qui correspondent aux toPersonName extrait de value sont en principe  contenus dans Namedetail. Pour les retrouver, on fait une requête sur les name de Namedetail qui correspondent aux toPersonName.
Les id correspondant aux toPersonName ne sont pas tous présents dans la table NameDetail. Or Namedetail est un sous échantillon de NameAll.
Donc si les id sont présents dans NameDetail(not null) alors on les prend sinon (si il n'y a pas de correspondance) on selectionne dans la table NameAll les id manquant qui correspondent aux toPersonName. On aujoute un first à select car il s'agit d'identifiant (unique).

In [ ]:
%%sql

create or replace temp view NameTrivia as 
select n.id, n.name,
n.value as triviaText,
trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1)) as toPersonName,
-- recherche de l'id dans Namedetail sinon Nameall
if ((select first(n2.id) from NameDetail n2 where n2.name = trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1))) is not null,
    (select first(n2.id) from NameDetail n2 where n2.name = trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1))), 
   (select first(na.id) from NameAll na where trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1)) = na.name))
   as toPersonId

from NameDetail n
where n.property = 'trivia' and n.value like '%(qv)%';

select * from NameTrivia;

,id,name,triviaText,toPersonName,toPersonId
0,2555649,"Spears, Britney",June 2001: Two Texas DJs created a panic for h...,*NSYNC,113.0
1,104362,"Bass, Lance","Was the last member to join '*NSYNC' (qv), aft...",*NSYNC,113.0
2,104362,"Bass, Lance",Performed with '*NSYNC' (qv) in the 1998 and 1...,*NSYNC,113.0
3,104362,"Bass, Lance",Member of boy group '*NSYNC' (qv).,*NSYNC,113.0
4,104362,"Bass, Lance",(November 1999) Performed with '*NSYNC' (qv) o...,*NSYNC,113.0
...,...,...,...,...,...
95,997263,"Matsui, Hideki","Is nicknamed ""Godzilla"" because of his power-h...",Akira Ifukube,NaN
96,674494,"Hill, Al",Father of 'Al Hill Jr.' (qv),Al Hill Jr.,NaN
97,1560084,"Tissier, Jean",Uncle of 'Alain Tissier' (qv).,Alain Tissier,NaN
98,2046426,"Gibbs, Marla",Is a best friend of the late 'Alaina Reed-Hall...,Alaina Reed-Hall,NaN


#### Question : 5 Lien IMDB-YAGO

On suppose que certaines personnes sont présentes à la fois dans IMDB et dans YAGO mais elles ne sont pas structurées de la même façon.

5a) Compléter les Personnes existant dans NameDetail avec des informations venant de yago (voir **yagoPerson** et **actorFacts**). 
Pour cela, définir la relation qui donne le lieu de naissance et le conjoint (ou la conjointe) d'une personne :

  - **NameYago** (id, name, yagoName, yagoBirthPlace, yagoSpouse) 

5b) Compléter le contenu de NameYago pour les personnes de IMDB existant dans *name_full*.

# Q5: Preparation NameYago
Pour construire la Table NameYago:

-
On crée d'abord des tables YagoPerson et ActorFacts en python puis en sql YagoPersonTable et ActorFactsTable. Pour simplifier les écritures, on extrait avec regexp_extract() les données variantes des liens . 

Par exemple, pour <http://yago-knowledge.org/resource/Heywood_L._Edwards> et <http://yago-knowledge.org/resource/Ramanichandran> on extrait Heywood_L._Edwards et Ramanichandran car le chemin <http://yago-knowledge.org/resource/> est invariant.

La table YagoPersonTable contient des noms de personnes et ActorFactsTable contient des noms avec des informations supplémentaires correspondantes : property correspond à la nature de la donnée ( exemples 'birthplace' et 'spouse')  et value à la valeur.





In [ ]:
#on cree des tables  python
yagoPerson.createOrReplaceTempView("YagoPerson")
actorFacts.createOrReplaceTempView("ActorFacts")


In [ ]:
%%sql
--tables Sql
--exctraction des valeurs dans les liens
--yagoPersonTable
create or replace temp view YagoPersonTable as
select regexp_extract(subject, '<http://yago-knowledge.org/resource/([^>]*)>') as name
from yagoPerson;

-- ActorFactsTable
create or replace temp view ActorFactsTable as
select regexp_extract(subject, '<http://yago-knowledge.org/resource/([^>]*)>') as name,
regexp_extract(property, '<http://schema.org/([^>]*)>') as property,
regexp_extract(object, '<http://yago-knowledge.org/resource/([^>]*)>') as value
from actorFacts;



""


#Q5:NameYago 
On cherche à afficher l'id, le nom, la date de naissance et le nom du conjoint des personnes contenus dans NameYago et Namedetails.

Pour cela :

1)Requêtes sur ActorFactsTable: On selectionne dans b les noms et valeurs correspondant à la propriété 'birthPlace'. De même pour 'spouse' (conjoint) dans s.

-
2)On fait une jointure entre les tables Namedetail et ActorFactsTable sur les noms qui sont dans Namedetail et dans ActorFactsTable avec la propriété 'birthPlace' ou 'spouse'.

3) On selectionne sur cette jointure les id et noms de Namedetails, les noms de Yago (YagoName) ainsi que leur date de naissance (b.place) ou nom de conjoint.(s.spouse) de ActorFactsTable. 

5b) On execute le même code avec Nameall, pour compléter les colonnes du  tableau NameYago avec des noms n'appartenant pas à NameDetail.


In [ ]:
%%sql

create or replace temp view NameYago as
select distinct n.id, n.name,
 b.name as yagoName, b.place as yagoBirthPlace,
 s.spouse as yagoSpouse
from NameDetail n,
--lieu de naissance
(select name, value as place from ActorFactsTable where property = 'birthPlace') as b,
-- conjoint
(select name, value as spouse from ActorFactsTable where property = 'spouse') as s

where n.name = b.name and n.name = s.name;

select * from NameYago;

,id,name,yagoName,yagoBirthPlace,yagoSpouse
0,716114,Ice-T,Ice-T,"Newark,_New_Jersey",Coco_Austin
1,2208688,Kyla,Kyla,"Tondo,_Manila",Rich_Alvarez
2,2298105,Maïwenn,Maïwenn,Les_Lilas,Luc_Besson
3,2524210,Shabnam,Shabnam,Dhaka,Robin_Ghosh
4,2524329,Shadia,Shadia,Cairo,Emad_Hamdy
5,2524329,Shadia,Shadia,Cairo,Salah_Zulfikar
6,65736,Asrani,Asrani,Jaipur,Manju_Asrani
7,1895700,Ciara,Ciara,"Austin,_Texas",Russell_Wilson
8,1904438,Colette,Colette,Saint-Sauveur-en-Puisaye,Henry_de_Jouvenel
9,1904438,Colette,Colette,Saint-Sauveur-en-Puisaye,Henry_Gauthier-Villars


In [ ]:
%%sql
--complète avec donnees Nameall
create or replace temp view NameYago as
select distinct n.id, n.name,
 b.name as yagoName, b.place as yagoBirthPlace,
 s.spouse as yagoSpouse
from NameAll n,
--lieu de naissance
(select name, value as place from ActorFactsTable where property = 'birthPlace') as b,
-- conjoint
(select name, value as spouse from ActorFactsTable where property = 'spouse') as s
where n.name = b.name and n.name = s.name;

select * from NameYago;

,id,name,yagoName,yagoBirthPlace,yagoSpouse
0,442402,Eita,Eita,Niigata_Prefecture,Kaela_Kimura
1,1997731,Fairuz,Fairuz,Beirut,Assi_Rahbani
2,3251095,Fairuz,Fairuz,Beirut,Assi_Rahbani
3,3687536,Fairuz,Fairuz,Beirut,Assi_Rahbani
4,716114,Ice-T,Ice-T,"Newark,_New_Jersey",Coco_Austin
...,...,...,...,...,...
95,1886338,Charo,Charo,Molina_de_Segura,Xavier_Cugat
96,1886339,Charo,Charo,Molina_de_Segura,Xavier_Cugat
97,1889032,Cher,Cher,"El_Centro,_California",Gregg_Allman
98,1889032,Cher,Cher,"El_Centro,_California",Sonny_Bono


### Exercice 2 : Analyse

#### Question 6 : Restructuration
L'objectif de cette question est de savoir définir une structure complexe contenant des attributs qui ne sont pas des valeurs simples mais les listes, des tuples, des listes de tuples, ...

Définir une table **Film** (id, titre, roles, paysSalles) avec 
 - *roles* est une liste de tuples (acteur, nom, rang) où rang est le numéro d'ordre du rôle (attribu *nr_order*)
 - *paysSalles* est la liste des pays où le film est sorti en salles, triée par revenu dans le pays

# Q6: Role (Movie_Id,acteur, nom, rang)

On cherche à réunir dans la liste roles :

- l'identifiant id de l'acteur contenu dans NameDetail

- le nom du rôle , role contenu dans Role_Type

- le numéro d'ordre du rôle, nr_order dans Cast_Info

Pour cela on effectue il faut effectuer 2 jointures: 

- une jointure entre Namedetail et Cast_info sur identité de la personne Cast_info.person_id=Namedetail.id

- une jointure entre Role_Type et Cast_Info sur l'identité du rôle : Cast_Info.role_id=Role_Type.id

On collecte les trois informations en liste de truplets avec collect_list() dans roles que l'on distinct pour éliminer les doublons, et on affiche l'identifiant du film qui nous servira lors des jointures pour le Film.






In [ ]:
%%sql
create or replace temp view Role as 
select c.movie_id, collect_list(distinct (n.id, r.role,c.nr_order)) as roles
from NameDetail n, Role_Type r, Cast_Info c
where n.id = c.person_id and c.role_id = r.id
group by c.movie_id;

select * from Role

,movie_id,roles
0,434,"[(1770691, actress, 7), (423423, actor, None),..."
1,1680,"[(1084034, producer, None)]"
2,1841,"[(1053812, actor, None)]"
3,2977,"[(1658620, actor, None), (987079, editor, None..."
4,3228,"[(1744984, actress, 1), (2173288, actress, Non..."
...,...,...
95,92630,"[(3052280, cinematographer, None)]"
96,95916,"[(2160973, actress, None), (1594413, actor, 1)]"
97,97539,"[(692946, actor, None), (1494829, actor, None)..."
98,99302,"[(1106974, actor, None), (1346820, actor, None..."


# Q6: Pays

Les pays sont contenus dans Movie_Info dans la chaine de caractères info (que l'on extrait à l'aide de regexp_extract()), avec la caractéristique info_type_id=16.

Pour associer le pays à chaque film on effectue une jointure entre Movie_Info et Cast_Info sur l'identifiant du film: 
Cast_Info.movie_id= Movie_Info.movie_id


In [ ]:
%%sql
create or replace temp view Pays as
select c.movie_id,  collect_list(distinct regexp_extract(m.info, '([^:]+)')) as paySalles
from Cast_Info c, Movie_Info m
where m.movie_id = c.movie_id and m.info_type_id=16
group by c.movie_id;

select * from Pays

,movie_id,paySalles
0,434,[Netherlands]
1,1680,[USA]
2,1841,[Spain]
3,2977,[USA]
4,3228,[USA]
...,...,...
95,86268,[USA]
96,87411,[Spain]
97,88256,[Spain]
98,89007,[Spain]


#Q6: Film (id, titre, roles, paySalles)

On cherche à réunir dans la table Film :

- le titre (title) du film contenu dans TitleDetail 

- la liste roles de Role

- le Pays dans lequelles le film a été diffusé PaySalles dans Pays
-l'identifiant du film id contenu dans toutes les tables précédentes ,( élément essentiel pour les jointures)


Pour cela  il faut effectuer 3 jointures: 

- une jointure entre TitleDetail et Cast_info sur l'identifiant du film Cast_info.movie_id=TitleDetail.id

- une jointure entre Role et Cast_info sur l'identifiant du film Cast_info.movie_id=Role.movie_id

- une jointure entre Pays et Cast_info sur l'identifiant du film Cast_info.movie_id=Pays.movie_id



In [ ]:
%%sql

create or replace temp view Film as
select distinct c.movie_id as id, t.title as titre, r.roles, p.paySalles as paysSalles
from  Role r, Pays p, Cast_Info c, TitleDetail t
where c.movie_id = t.id and c.movie_id = r.movie_id and c.movie_id = p.movie_id;

select * from Film 

,id,titre,roles,paysSalles
0,434,Brand in Mokum,"[(1770691, actress, 7), (423423, actor, None),...",[Netherlands]
1,1680,A Tiny Problem in the Kitchen,"[(1084034, producer, None)]",[USA]
2,1841,(2005-03-20),"[(1053812, actor, None)]",[Spain]
3,2977,Death: The Gift That Lasts Forever,"[(1658620, actor, None), (987079, editor, None...",[USA]
4,3228,He Followed Me Home/Love 'Em and Flea 'Em,"[(1744984, actress, 1), (2173288, actress, Non...",[USA]
...,...,...,...,...
95,95916,(1975-05-27),"[(2160973, actress, None), (1594413, actor, 1)]",[Spain]
96,97539,The Fine Art of Separating People from Their M...,"[(692946, actor, None), (1494829, actor, None)...",[UK]
97,99302,Tunnel to Freedom,"[(1106974, actor, None), (1346820, actor, None...",[USA]
98,99758,(2007-10-27),"[(2408215, actress, 3), (1257323, actor, None)]",[Spain]


#### Question 7 : Fenêtres
On considère la table **RevenuWeekEndFilm**(id, titre, pays, jour, mois, année, revenu, écrans) dont les informations proviennent de la propriété *weekend gross* de TitleDetail
* un week-end est défini par les attributs (jour, mois, année)
* l'attribut *écrans* correspond à la quantité associée au terme *screens*

Exprimer les requêtes :
 - 7a) Pour chaque mois, le revenu total des films sortis aux USA durant les 3 derniers mois. Le schéma du résultat est : 
     - (mois, année, revenuMoisCourant, revenu3DerniersMois)
 - 7b) Pour chaque pays, le classement des films par revenu total. Le schéma du résultat est :
      - (pays, id, titre, revenuTotal, classement)
 - 7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film. Le schéma du résultat est :
     - (id, titre, revenuAnnuel, annee, mois, jour, revenuWeekEnd, numéroPériode)

In [ ]:
%%sql
select *
from TitleDetail
where property = 'weekend gross';

,id,title,production_year,kind,property,value
0,1640952,39 Pounds of Love,2005,movie,weekend gross,$90 (USA) (22 January 2006) (1 screen)
1,1640952,39 Pounds of Love,2005,movie,weekend gross,"$1,227 (USA) (15 January 2006) (1 screen)"
2,1640952,39 Pounds of Love,2005,movie,weekend gross,$160 (USA) (18 December 2005)
3,1640952,39 Pounds of Love,2005,movie,weekend gross,"$6,532 (USA) (11 December 2005)"
4,1640952,39 Pounds of Love,2005,movie,weekend gross,"$8,055 (USA) (4 December 2005) (1 screen)"
...,...,...,...,...,...,...
95,1674846,Aliens of the Deep,2005,movie,weekend gross,"$2,773 (USA) (1 April 2007) (1 screen)"
96,1674846,Aliens of the Deep,2005,movie,weekend gross,"$4,694 (USA) (18 March 2007) (1 screen)"
97,1674846,Aliens of the Deep,2005,movie,weekend gross,"$2,789 (USA) (11 March 2007)"
98,1674846,Aliens of the Deep,2005,movie,weekend gross,"$2,488 (USA) (4 March 2007)"


#Préparation question 7: 
RevenuWeekEndFilm(id, titre, pays, jour, mois, année, revenu, écrans)

On extrait presque toutes les informations contenues dans TitleDetail avec la prorpiété 'weekend gross'.
On prend la colonne id,title,et les informations contenue dans values.


In [ ]:
%%sql

-- RevenuWeekEndFilm
create or replace temp view RevenuWeekEndFilm as
select id, title as titre,
regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 1) as devise,
regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 3) as pays,
to_date(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2} [a-zA-Z]* [0-9]{0,4})\\)', 4), 'd MMMM yyyy') as date,
cast(regexp_replace(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 2),',','') as int) as revenu,
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 4) as int) as jour,
month(to_date(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 5), "MMMM")) as mois,
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 6) as int) as annee,
-- Il n'ya pas toujours de screens
if (regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\) \\(([0-9]+) screen\\)', 7) != '',
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\) \\(([0-9]+) screen\\)', 7) as int), 
0) as ecrans
from TitleDetail
where property = 'weekend gross';

select *
from RevenuWeekEndFilm

,id,titre,devise,pays,date,revenu,jour,mois,annee,ecrans
0,1640952,39 Pounds of Love,$,USA,2006-01-22,90,22,1,2006,1
1,1640952,39 Pounds of Love,$,USA,2006-01-15,1227,15,1,2006,1
2,1640952,39 Pounds of Love,$,USA,2005-12-18,160,18,12,2005,0
3,1640952,39 Pounds of Love,$,USA,2005-12-11,6532,11,12,2005,0
4,1640952,39 Pounds of Love,$,USA,2005-12-04,8055,4,12,2005,1
...,...,...,...,...,...,...,...,...,...,...
95,1674846,Aliens of the Deep,$,USA,2007-04-01,2773,1,4,2007,1
96,1674846,Aliens of the Deep,$,USA,2007-03-18,4694,18,3,2007,1
97,1674846,Aliens of the Deep,$,USA,2007-03-11,2789,11,3,2007,0
98,1674846,Aliens of the Deep,$,USA,2007-03-04,2488,4,3,2007,0


# Q 7.A
- (mois, année, revenuMoisCourant, revenu3DerniersMois)
1)revenuMoisCourant

On selectionne le mois de la date et  l'année de la date.
Ensuite on regroupe les revenus dans le pays "USA" (where pays= 'USA') par année et par mois (group by year(date), month(date)) et on somme sur les revenus de ces nouveaux groupes (sum(revenu)), ce qui donne le revenu total des films aux US du mois courant, revenuMoisCourant




In [ ]:
%%sql

create or replace temp view T_revenuMoisCourant as 
select year(date) as annee, month(date) as mois, sum(revenu) as revenuMoisCourant
from RevenuWeekEndFilm
where pays = 'USA'
group by year(date), month(date)
order by year(date), month(date);

select * from T_revenuMoisCourant;

,annee,mois,revenuMoisCourant
0,1977,10,1540635
1,1982,1,1955016
2,1982,2,1160714
3,1984,2,58689
4,1984,3,1803432
...,...,...,...
95,2009,4,995797
96,2009,5,101398
97,2009,6,17498316
98,2009,7,5549575


# Q 7.B
(pays, id, titre, revenuTotal, classement)

On calcule d'abord le revenuTotal en sommant les revenus par pays par film.
Ensuite on trie et on classe le revenuTotal des films dans l'ordre décroissant dans chaque pays que l'on affiche dans une colonne classement. Donc dans chaque pays les films sont ordonnés par revenu Total décroissant. Par exemple le classement 1 correspond au film au revenu le plus fort dans un pays donné.
 

In [ ]:
%%sql
with T as (
-- revenu total 
select pays, id, titre, sum(revenu) as revenuTotal
from RevenuWeekEndFilm
group by pays, id, titre
order by pays, id, titre, sum(revenu) desc)

-- classement
select pays, id, titre, revenuTotal, rank() over (partition by pays order by revenuTotal desc) as classement
from T
order by pays, classement

,pays,id,titre,revenuTotal,classement
0,Argentina,1890399,Final Destination 2,387753,1
1,Argentina,1916458,Ghost Dog: The Way of the Samurai,193407,2
2,Australia,1916458,Ghost Dog: The Way of the Samurai,697828,1
3,Australia,2294518,Slumdog Millionaire,409361,2
4,Austria,1916458,Ghost Dog: The Way of the Samurai,3445077,1
...,...,...,...,...,...
95,USA,1961342,How the Garcia Girls Spent Their Summer,91432,35
96,USA,2233578,Raye makhfi,77787,36
97,USA,2409025,The Romantics,77396,37
98,USA,1791664,Crackers,58689,38
